Preparing the dataframe for training

In [9]:
import numpy as np
import pandas as pd
x_train = np.array([
    [0,0],
    [1,1],
    [1,0],
    [0,1]
])
y_train = np.array([
    [0],
    [1],
    [0],
    [0]
])
# x_train.shape
# y_train.shape
#Combining the features and respective outputs
data = np.hstack((x_train,y_train))
#Converting it into dataframe 
df = pd.DataFrame(data, columns=["x1","x2","y"])
print(df)

   x1  x2  y
0   0   0  0
1   1   1  1
2   1   0  0
3   0   1  0


Preparing the weight matrix for Wxh (between input and hidden layer) and Who (between hidden and output layer)

In [10]:
#defining the number of neurons in each layer to make it more systematic. The ANN consists of 3 layers, in input layer : 2, hidden layer : 3, output layer : 1 neurons respectively
x_n = 2
h_n = 3
o_n = 1

#defining the weight matrix 
np.random.seed(42)
Wxh = np.random.randn(x_n,h_n)
Who = np.random.randn(h_n,o_n)
print(Wxh)

[[ 0.49671415 -0.1382643   0.64768854]
 [ 1.52302986 -0.23415337 -0.23413696]]


Equations for forward propagation 

In [11]:
def sigmoid(x):
    pass

In [12]:
ua_h = x_train @ Wxh         #shape = (4,3)

a_h = 1/(1 + np.exp(ua_h))   #shape = (4,3)
# print(a_h)
ua_o = a_h @ Who             #shape = (4,1)

output = 1/(1 + np.exp(ua_o))   #shape = (4,1)
print(output)

[[0.28118594]
 [0.38877303]
 [0.3001949 ]
 [0.38958691]]


Cost error used here : Mean Squared Error

In [13]:
sq_term = np.square(y_train - output)
mse = np.mean(sq_term)
